## PSV to DF

This notebook loads each PSV file from the Physionet database and concatenates all the records into a single dataframe object. Note that a new patient ID row is created, which is based on the filename. This enables downstream analysis to distinguish between subjects.

The expected file format is a folder called "training_setA" and "training_setB" which each contain 20000 files. Running this script is optional as the final dataframe has been uploaded to google drive and is automatically downloaded in the feature_engineering notebook

Import libraries

In [6]:
import pandas as pd
import pdb
import numpy as np
import glob
import os

In [7]:
#get a list of all the files
files1 = glob.glob('/Users/zhaoxiuheng/Desktop/课程/Cognitive AI/project/code/training_setA/*.psv')
files2 = glob.glob('/Users/zhaoxiuheng/Desktop/课程/Cognitive AI/project/code/training_setB/*.psv')
files = np.concatenate((files1, files2))
sorted_files = sorted(files, key=lambda x: os.path.basename(x))

# print(sorted_files)

# print(files)

df_list = []
for ind, f in enumerate(sorted_files):
    patient_id = f.split('/')[-1].split('.')[0]
    # print(patient_id)
    df = pd.read_csv(f, sep='|')
    df = df.assign(patient=patient_id)

    #redefine the labels to be 1 for t >= t_sepsis
    #in other words, a label of 1 now means that sepsis has occurred in this window
    #in practice, what this means is set the first six 1 labels to 0
    df.loc[df[df['SepsisLabel'] == 1].head(6).index.values, 'SepsisLabel'] = 1
    
    #print a status update
    if ind % 1000 == 0:
        print(ind)
    
    #append the current parsed file to the list 
    df_list.append(df)


#save all the loaded files into a pickle file
df = pd.concat(df_list)
df = df.reset_index(drop=True)
df.to_pickle('combined.pkl')


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000


In [8]:
df.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,patient
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,83.14,0,NaN,NaN,-0.03,1,0,p000001
1,97.0,95.0,NaN,98.0,75.33,NaN,19.0,NaN,NaN,NaN,...,NaN,NaN,83.14,0,NaN,NaN,-0.03,2,0,p000001
2,89.0,99.0,NaN,122.0,86.00,NaN,22.0,NaN,NaN,NaN,...,NaN,NaN,83.14,0,NaN,NaN,-0.03,3,0,p000001
3,90.0,95.0,NaN,NaN,NaN,NaN,30.0,NaN,24.0,NaN,...,NaN,NaN,83.14,0,NaN,NaN,-0.03,4,0,p000001
4,103.0,88.5,NaN,122.0,91.33,NaN,24.5,NaN,NaN,NaN,...,NaN,NaN,83.14,0,NaN,NaN,-0.03,5,0,p000001
